In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [4]:
url_data ="https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}&cat=FoulsCommited&agg=PerGame"
years = list(range(2001, 2022))

In [6]:
for year in years:
    with open("fouls_commited_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        url = url_data.format(year)
        data = requests.get(url)
        file.write(data.text)

In [8]:
dfs = []
for year in years:
    with open("fouls_commited_htmls/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed")
    player_table = soup.find_all(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")[0]
    fouls_commited = pd.read_html(str(player_table))[0]
    fouls_commited["Year"] = year
    dfs.append(fouls_commited)

In [9]:
fouls_commited= pd.concat(dfs)
fouls_commited

,Rank,Player,Team,GamesG,Fouls CommitedFL-,Ave.,Per 40 min.40m,Year
0,1,"GRUM, SAMOGRUM, SAMO",MESKRKA Novo MestoKRKA Novo Mesto,14,62,4.43,8.28,2001
1,2,"LOKHMANCHUK, ALEXANDERLOKHMANCHUK, ALEXANDER",FRAOpel SkylinersOpel Skyliners,13,57,4.38,7.90,2001
2,3,"PRASKEVICIUS, VIRGINIJUSPRASKEVICIUS, V.",OOSTelindus OostendeTelindus Oostende,14,61,4.36,5.89,2001
3,4,"OBERTO, FABRICIOOBERTO, F.",BKNTau CeramicaTau Ceramica,19,78,4.11,6.41,2001
4,5,"ROBINSON, RUMEALROBINSON, R.","2 ZADZadarZadar,",12,48,4.00,5.72,2001
...,...,...,...,...,...,...,...,...
15,16,"LAPROVITTOLA, NICOLASLAPROVITTOLA, N.",BARFC BarcelonaFC Barcelona,38,98,2.58,5.05,2021
16,17,"VESELY, JANVESELY, J.",FBBFenerbahce Beko IstanbulFenerbahce Beko Ist...,22,56,2.55,3.57,2021
17,18,"MACON JR, DARYLMACON, D.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,31,78,2.52,3.89,2021
18,19,"MICIC, VASILIJEMICIC, V.",EFSAnadolu Efes IstanbulAnadolu Efes Istanbul,34,85,2.50,3.31,2021


In [10]:
engine = create_engine('sqlite:///commited_fouls_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
fouls_commited.to_sql(
    name = "commited_fouls_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

847

In [12]:
commited_fouls_leaders_select = session.execute('select * from "commited_fouls_leaders"')
commited_fouls_leaders_select.all()

[(0, 1, 'GRUM, SAMOGRUM, SAMO', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 62, 4.43, 8.28, 2001),
 (1, 2, 'LOKHMANCHUK, ALEXANDERLOKHMANCHUK, ALEXANDER', 'FRAOpel SkylinersOpel Skyliners', 13, 57, 4.38, 7.9, 2001),
 (2, 3, 'PRASKEVICIUS, VIRGINIJUSPRASKEVICIUS, V.', 'OOSTelindus OostendeTelindus Oostende', 14, 61, 4.36, 5.89, 2001),
 (3, 4, 'OBERTO, FABRICIOOBERTO, F.', 'BKNTau CeramicaTau Ceramica', 19, 78, 4.11, 6.41, 2001),
 (4, 5, 'ROBINSON, RUMEALROBINSON, R.', '2 ZADZadarZadar,', 12, 48, 4.0, 5.72, 2001),
 (5, 5, 'VUJANIC, MILOSVUJANIC, M.', 'PARPartizanPartizan', 12, 48, 4.0, 5.74, 2001),
 (6, 7, 'GNJIDIC, BORISGNJIDIC, B.', 'MESKRKA Novo MestoKRKA Novo Mesto', 14, 55, 3.93, 5.52, 2001),
 (7, 8, 'PANOV, SERGEYPANOV, S.', 'PEMUral GreatUral Great', 20, 78, 3.9, 6.49, 2001),
 (8, 9, 'MENEGHIN, ANDREAMENEGHIN, A.', 'FORSkipper BolognaSkipper Bologna', 19, 74, 3.89, 5.66, 2001),
 (9, 10, 'JEROME, JOHNJEROME, JOHN', 'CHASpirou BasketSpirou Basket', 14, 54, 3.86, 6.4, 2001),
 (10, 10, '

In [ ]:
X = fouls_commited[[]]